In [ ]:
import torch
from sklearn.datasets import fetch_california_housing
from src.mssp import MSSP

In [ ]:
data = fetch_california_housing()
X = torch.tensor(data.data, dtype=torch.double)
y = torch.tensor(data.target, dtype=torch.double)

In [ ]:
RANDOM_SEED = 42
if RANDOM_SEED is not None:
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)


In [ ]:
i = torch.randperm(len(X))
i_train = i[:int(len(X)*0.8)]
i_valid = i[int(len(X)*0.8):int(len(X)*0.9)]
i_test = i[int(len(X)*0.9):]

In [ ]:
X_train, y_train = X[i_train], y[i_train]
X_valid, y_valid = X[i_valid], y[i_valid]
X_test, y_test = X[i_test], y[i_test]

In [ ]:
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

In [ ]:
model = MSSP(
    n_best=120, 
    loss_fn="mse", 
    random_seed=RANDOM_SEED, 
    epochs=10, 
    diversity_ratio=0.75, 
    pow_cross=True,
    # cv=5
)

In [ ]:
# model.fit(X_train, y_train, X_valid, y_valid)
model.fit(X_train, y_train)


In [ ]:
print(model.evaluate(X_test, y_test, top_k=32))
print(model.evaluate(X_test, y_test, top_k=16))
print(model.evaluate(X_test, y_test, top_k=8))
print(model.evaluate(X_test, y_test, top_k=4))
print(model.evaluate(X_test, y_test, top_k=1))

# (tensor(0.3946, dtype=torch.float64), tensor(2.2155, dtype=torch.float64))
# (tensor(0.3940, dtype=torch.float64), tensor(2.2076, dtype=torch.float64))
# (tensor(0.3941, dtype=torch.float64), tensor(2.2084, dtype=torch.float64))
# (tensor(0.3940, dtype=torch.float64), tensor(2.2077, dtype=torch.float64))
# (tensor(0.3942, dtype=torch.float64), tensor(2.2090, dtype=torch.float64))
# (tensor(0.3946, dtype=torch.float64), tensor(2.2127, dtype=torch.float64))

In [ ]:
model.model[0].head.cross